In [7]:
import numpy as np
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister
from qiskit import BasicAer
from qiskit import execute
import math
%matplotlib inline

In [8]:
def prepare_swap_circuit(theta_true,phi_true,theta_guess,phi_guess):
    """
    This function constructs and return a quantum circuit to do a swap test based on:
    1, a set of randomly chosen theta and phi value
    2, a set of guessed theta and phi value
    """
    anc = QuantumRegister(1,'ancilla')
    qr1 = QuantumRegister(1,'q1')
    qr2 = QuantumRegister(1,'q2')
    cr = ClassicalRegister(1,'c')
    qc = QuantumCircuit(qr1,qr2,anc,cr)
    
    # qr1 is set by a set of randomly chosen theta and phi
    qc.ry(theta_true,qr1)
    qc.rz(phi_true,qr1)
    
    # qr2 is a guess of theta and phi
    qc.ry(theta_guess,qr2)
    qc.rz(phi_guess,qr2)
    
    # Compare qr1 and qr2 by swap circuit
    qc.h(anc)
    qc.cswap(anc,qr1,qr2)
    qc.h(anc)
    
    qc.measure(anc,cr)
    return qc

In [15]:
# Randomly choose theta and phi
theta_true = np.random.uniform()
phi_true = np.random.uniform()

# Grid range for theta
theta_range = [0.,1.]
# Number of grid points scanned for theta
n_theta = 50

# Grid range for phi
phi_range = [0.,1.]
# Number of grid points scanned for phi
n_phi = 50

# Number of times to run the circuit
shots = 50000

print("True value: ",theta_true,phi_true)
out_list = []
for it in range(n_theta):
    for ip in range(n_phi):
        
        # Calculate the guess values for theta and phi
        theta_guess = (theta_range[1]-theta_range[0]) / n_theta * it + theta_range[0]
        phi_guess = (phi_range[1]-phi_range[0]) / n_phi * ip + phi_range[0]
        
        # Prepare quantum circuit for the swap test
        qc = prepare_swap_circuit(theta_true,phi_true,theta_guess,phi_guess)
        
        # Run the swap test n times
        backend = BasicAer.get_backend('qasm_simulator')
        results = execute(qc, backend=backend, shots=shots).result()
        answer = results.get_counts()
        
        # Calculate swap result
        sumM = float(answer['1']) if '1' in answer else 0.
        s = 1.-2./shots*sumM
        
        # Save result of this grid point to the list
        out_list.append([theta_guess,phi_guess,s])
        
# Print the grid point with max s
out_list.sort(key=lambda x: x[2], reverse=True)
print(out_list[0])

True value:  0.8587358512674191 0.7710368591672467
[0.86, 0.76, 1.0]
